<div class="alert alert-block alert-info">
LINMA2415 - Project - Question 9

- Colson Adeline 19752000
    
- Lhernould Benoit 77102200 

- El Bared Mary Jo 48212000 
  
    
</div>

<div class="alert alert-block alert-info">
Load Data
</div>

In [1]:
using CSV, DataFrames, Dates, HiGHS, JuMP

In [2]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

8760-element Vector{Float64}:
 56250.0
 54300.0
 53600.0
 50000.0
 47100.0
 45850.0
 45850.0
 46000.0
 46400.0
 46950.0
     ⋮
 61450.0
 60150.0
 62500.0
 65600.0
 65000.0
 62300.0
 59850.0
 59700.0
 63750.0

In [3]:
file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)

file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)

8760×1 Matrix{Float64}:
 0.12594
 0.123693
 0.124232
 0.13045
 0.140045
 0.136766
 0.129185
 0.135791
 0.152821
 0.14136
 ⋮
 0.28469
 0.31118
 0.31791
 0.320436
 0.322001
 0.312379
 0.272079
 0.214026
 0.166168

<div class="alert alert-block alert-info">
Constant and data
</div>

In [4]:
# Constants and Data
hours_per_year = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20

# A=[1299.44,634.83,459.12,437.52,812.53,237.66]
# EA=[175.2252157,85.60445144,61.91153839]

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6),
    "Onshore Wind" => (capex=700*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=58.99797384*1000, price =0, C=0, emissions=0.0),
    "Offshore Wind" => (capex=1300*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=109.5676657*1000, price =0, C=0, emissions=0.0),
    "PV" => (capex=400*1000, om=0.03, debt_ratio=0.8, heat_rate=0, EA=32.04823387*1000, price =0, C=0, emissions=0.0)
)

Dict{String, NamedTuple{(:capex, :om, :debt_ratio, :heat_rate, :EA, :price, :C, :emissions)}} with 6 entries:
  "Coal"          => (capex = 2000000, om = 0.03, debt_ratio = 0.62, heat_rate …
  "CCGT"          => (capex = 950000, om = 0.03, debt_ratio = 0.56, heat_rate =…
  "Offshore Wind" => (capex = 1300000, om = 0.03, debt_ratio = 0.7, heat_rate =…
  "OCGT"          => (capex = 700000, om = 0.03, debt_ratio = 0.6, heat_rate = …
  "PV"            => (capex = 400000, om = 0.03, debt_ratio = 0.8, heat_rate = …
  "Onshore Wind"  => (capex = 700000, om = 0.03, debt_ratio = 0.7, heat_rate = …

<div class="alert alert-block alert-info">
Question 9
</div>

In [5]:
model = Model(HiGHS.Optimizer)

@variable(model, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model, p[g in keys(technologies), t=1:hours_per_year] >= 0)  # Hourly production
@variable(model, 0<=d[t=1:hours_per_year])


@objective(model,Min,sum((technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g]+sum(technologies[g].C*p[g,i] for i in 1:hours_per_year)  for g in keys(technologies))-sum(lambda*d[i] for i in 1:hours_per_year))


@constraint(model, price[t=1:hours_per_year], sum(p[g, t] for g in keys(technologies)) == d[t])
@constraint(model,[i in 1:hours_per_year],d[i] <= D_t[i])


# New constraint on capacity
for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model, [t=1:hours_per_year], p[g, t] <= k[g])
end
for t in 1:hours_per_year
    @constraint(model, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model, p["PV", t] <= capacity_pv[t]*k["PV"])
end

optimize!(model)

println("\n")
println("Point I")
println("Optimal capacities (kW):")
total_production = 0
production_by_tech = Dict(g => 0.0 for g in keys(technologies))
for g in keys(technologies)
    cap = value(k[g])  # This ensures 'cap' remains a floating-point number
    println("$(g): $(cap) MW")
    for t in 1:hours_per_year
        production_by_tech[g] += value(p[g, t])  # Accumulate as floating-point values
        total_production += value(p[g, t])  # Keep total production as floating-point
    end
end

println("\n")

println("Point II")
println("Share of each technology in effective production:")
for g in keys(technologies)
    share = production_by_tech[g] / total_production
    println("$(g): $(share * 100) %")
end
println("\n")


println("Point III")
println("Total Production: $(total_production) MWh")
total_cost = sum((technologies[g].EA+technologies[g].capex)*value.(k[g])+sum((technologies[g].C)*value.(p[g,i]) for i in 1:hours_per_year)  for g in keys(technologies))
println("Total Investment and Production Cost: €$(total_cost)")
println("\n")


println("Point IV")
# CO2 Emissions data
emissions = Dict(
    "Coal" => 1.4,
    "CCGT" => 0.5,
    "OCGT" => 0.6,
    "Onshore Wind" => 0.0,
    "Offshore Wind" => 0.0,
    "PV" => 0.0
)

# Calculating CO2 emissionys
total_co2_emissions = sum(emissions[g] * production_by_tech[g] for g in keys(production_by_tech))
println("\nTotal CO2 Emissions: $(total_co2_emissions) tons")
println("\n")


println("Point V")
load_curtailment = sum(max(0, D_t[t] - value(d[t])) for t in 1:hours_per_year)
println("Total Load Curtailment: $(load_curtailment) MWh")
println("\n")


println("Point VI")
profit = Dict(g => 0.0 for g in keys(technologies))
prices = [JuMP.dual(price[i]) for i in 1:hours_per_year]

for g in keys(technologies)
    profit[g] = -(technologies[g].EA + technologies[g].capex) * value(k[g]) - 
                 sum((technologies[g].C - prices[i]) * value(p[g, i]) for i in 1:hours_per_year)
end

for g in keys(technologies)
    println("Profit for technology $g: $(profit[g])")
end

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-03, 1e+00]
  Cost   [1e+01, 2e+05]
  Bound  [0e+00, 0e+00]
  RHS    [3e+04, 1e+05]
Presolving model
57214 rows, 57220 cols, 154122 nonzeros  0s
57214 rows, 57220 cols, 154122 nonzeros  0s
Presolve : Reductions: rows 57214(-12866); columns 57220(-4106); elements 154122(-16972)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
      36514    -2.3382648187e+12 Pr: 17989(8.64754e+07) 6s
      39811    -2.3369808138e+12 Pr: 3031(1.4813e+07) 11s
      40340    -2.3368471160e+12 Pr: 0(0) 12s
      40340    -2.3368471160e+12 Pr: 0(0) 12s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 40340
Objective value     : -2.3368471160e+12
HiGHS run time      :         11.74


Point I
Optimal capacities 